In [5]:

import sys
# sys.path.append("/home/jerryin/jupyter_proj/csgo/")
sys.path.append("../")

from Util.util import Constant
from Util import util
import time
import requests
import pickle
from Ip import ip
from Account.rawAccountPool import raw_accounts
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from Sms.sms import sms
from Account.Buyer.BuyerList import buyerlist
'''

2种账号：
    Buyer（购买者）账号和Viewer(浏览者), Viewer获取数据并记录可购买的商品，Buyer购买商品。

1. Account -> 可用于Buyer和Viewer 这两个类继承Account类
    Buyer和Viewer获取、更新ip方式相同
    Buyer和Viewer登录方式不同
    Buyer和Viewer获取账号方式不同
    Viewer直接从接口获取手机号，并通过手机号验证登录后，获取10天的免登录状态，用这10天cookies获取数据，若过期则直接换下一个手机号。
    Buyer通过excel或json获取账号和密码，通过密码登录，获取10天的免登录状态，用这10天的cookies购买商品，若过期则重新登录。
    登录完成后把账号存放于硬盘中。
2. AccountPool -> 一组Account的List，存放Account。
    从硬盘中读取一个个Account信息，并验证可用性。
    若账号不可用，则直接删除，若IP不可用，则用Account来更新IP。
'''
class Account(object):
    '''
    methods:
        1. isIpAlive: check the ip banded with the account is alive or not. return true or false.
        2. accountStatus: check the status of the account, 0: logined and market available. 1: did not login. -1: can not access market or other problems like account blocked problem.
        3. setCookieAndToken: process cookie map to cookie str and token str. return cookie, csrf_token.
        4. updateIp: update ip.
        5. getIp: return the new ip.
        6. blockPhone: None: block the phone
        7. login: None: login a new account.
    '''
    def __init__(self):
        # self.ip = self.getIp() # like 127.0.0.1:8080
        self.ip = '36.213.73.168:16816'
        self.BUFF_ACCOUNT = '17706619337'
        self.BUFF_PWD = 'Txjhnnb12345'
        self.STEAM_EMAIL = ''
        self.STEAM_ACCOUNT = ''
        self.STEAM_PWD =''
        self.cookie = ''
        self.csrf_token = '' # the token for visiting website
        self.status = -1

    def isIpAlive(self) -> bool:
        '''
            check the ip banded with the account is alive or not.
        '''
        return ip.checkIpAlive(self.ip)
    
    def accountStatus(self) -> int:
        '''
        return: 
            0: login and market available
            1: did not login
            -1: can not access market or other problems  like account blocked problem。
        '''
        try:
            if not self.isIpAlive():
                self.updateIp()

            headers = {
                ':method': 'GET',
                ':path': Constant.account_availability_check_re_addr,
                'cookie': self.cookie + 'csrf_token=' + self.csrf_token,
                'sec-fetch-user': '?1',
                'upgrade-insecure-requests': '1',
            }

            tmp_session = Constant.getSession() # get a tmp session
            tmp_session.proxies.update(Constant.getProxy(self.ip))
            
            resp = tmp_session.get(Constant.account_availability_check_ab_addr, headers = headers)
            # print(resp.json())
            code = resp.json()['code']
            print('check account status, the code is: ' + code + '\n')
            if code == 'OK' or code == 'ok':
                return 0
            elif code == 'Login Required':
                return 1
            else:
                return -1
        except Exception as e:
            # print(e)
            return -1
    
    def setCookieAndToken(self, cookieList: list) -> tuple: 
        '''
            process cookie map to cookie str and token str.
        '''
        cookie_map = cookieList
        cookie = ''
        deviceId = ''
        LocaleSupported = ''
        game = ''
        NTES_YD_SESS = ''
        S_INFO = ''
        P_INFO = ''
        remember_me = ''
        session = ''
        csrf_token = ''
        for _map in cookie_map:
            if(_map['name'] == 'csrf_token'):
                csrf_token = _map['value']
            elif(_map['name'] == 'Device-Id'):
                deviceId = 'Device-Id=' + _map['value'] + '; '
            elif(_map['name'] == 'Locale-Supported'):
                LocaleSupported = 'Locale-Supported=' + _map['value'] + '; '
            elif(_map['name'] == 'game'):
                game = 'game=' + _map['value'] + '; '
            elif(_map['name'] == 'NTES_YD_SESS'):
                NTES_YD_SESS = 'NTES_YD_SESS=' + _map['value'] + '; '
            elif(_map['name'] == 'S_INFO'):
                S_INFO ='S_INFO=' + _map['value'] + '; '
            elif(_map['name'] == 'P_INFO'):
                P_INFO = 'P_INFO=' + _map['value'] + '; '
            elif(_map['name'] == 'remember_me'):
                remember_me = 'remember_me=' + _map['value'] + '; '
            elif(_map['name'] == 'session'):
                session = 'session=' + _map['value'] + '; '
        cookie = deviceId + LocaleSupported + game + NTES_YD_SESS + S_INFO + P_INFO + remember_me + session
        return cookie, csrf_token
    
    def updateIp(self) -> None:
        '''
            update the ip banded with this account.
        '''
        self.ip = self.getIp()
    
    def getIp(self) -> str:
        '''
            return the new ip.
        '''
        return ip.getIp()
    
    def blockPhone(self) -> None:
        pass
    
    def login(self) -> None:
        pass

In [6]:
class Viewer(Account):
    '''
    methods:
        super from Account class:
        1. isIpAlive: check the ip banded with the account is alive or not. return true or false.
        2. accountStatus: check the status of the account, 0: logined and market available. 1: did not login. -1: can not access market or other problems  ip problem or account blocked problem.
        3. setCookieAndToken: process cookie map to cookie str and token str. return cookie, csrf_token.
        4. updateIp: update ip.
        5. getIp: return the new ip.
        
        self functions:
        1. login: login account. return true or false.
        2. blockPhone: None: block the phone
    '''
    
    def blockPhone(self) -> None:
        '''
            block the useless phone.
        '''
        print('block phone: ' + self.phone + ' block message is: ' + sms.blockPhone(self.phone))
    

    def login(self) -> bool:
        '''
            login the new or expired account, return the bool type as success or not.
            process:
                1. open the page and click the login/register button.
                2. wait 1.5s to open the login iframe page.
                3. click 10 days no login and the agreement.
                4. get the phone number and input into the phone input box automatically.
                5. click the send sms button, and wait 5 sec for the HUMAN VERIFICATION. (DO IT BY SELF!!!)
                6. if did not receive the sms after 45 sec, block the phone number and return false.
                7. click the login button and wait maximum 30 sec to login and get the cookies.
                8. check the account status, return true if logined or vice vera.
        '''
        try:

            options =  webdriver.ChromeOptions()

            options.add_experimental_option("detach", True) # make the chrome window always opened
            
            
            while True: # get the useful ip
                print(self.ip)
                options.add_argument(f"--proxy-server={self.ip}")
                # driver = webdriver.Chrome(executable_path= '/Users/yin/Desktop/chromedriver-mac-x64/chromedriver', options = options)
                driver = webdriver.Chrome(options = options)
                driver.get(Constant.ip_check_addr)
                try:
                    return_ip_addr = driver.find_element_by_tag_name('pre') # return 101.37.22.207
                    if return_ip_addr.text == self.ip.split(":")[0]:
                        print('IP ' + self.ip + ' can be use' + '\n')
                        driver.close()
                        break
                except BaseException as e:
                    driver.close()
                    options.arguments.remove(f"--proxy-server={self.ip}")
                    self.updateIp()
                    continue
            
            
            # driver = webdriver.Chrome(executable_path= '/Users/yin/Desktop/chromedriver-mac-x64/chromedriver', options = options)
            driver = webdriver.Chrome(options = options)

            driver.get(Constant.domain)
            try:
                driver.find_element_by_xpath('//div[@class="nav nav_entries"]/ul/li/a[@onclick="loginModule.showLogin()"]').click() # click login/register button
            except BaseException as e:
                return False

            try:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@id="j_login"]/iframe')))

                driver.switch_to.frame(driver.find_element_by_xpath('//div[@id="j_login"]/iframe')) # swith to the login iframe

                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//input[@id="phoneipt"]')))

            except BaseException as e:
                return False
            
            try:
                # driver.find_element_by_xpath('//div[@id="agree-checkbox"]/span/i').click() # click agreement
                driver.find_element_by_xpath('//input[@class="zc-un-login"]').click() # click agreement

                # driver.find_element_by_xpath('//input[@id="phoneipt"]').send_keys(self.BUFF_ACCOUNT)  # input phone number

                # driver.find_element_by_xpath('//input[@class="j-inputtext dlemail"]').send_keys(self.BUFF_PWD)  # input password

            except BaseException as e:
                return False
            
            code = []
            while(len(code) == 0): # did not receive the code verification
                self.phone = sms.getPhone() # get a new phone number
                # self.phone = '16537928820'
                print('buff phone number: ' + self.phone)
                try:
                    driver.find_element_by_xpath('//input[@class="dlemail j-nameforslide"]').send_keys(self.phone) 
                except BaseException as e:
                    return  False
                time.sleep(1) # time sleep for the human verification
                try:
                    driver.find_element_by_xpath('//div[@class="pcbtn f-fl"]/a').click() # send the sms code
                except BaseException as e:
                    return False
                _ind = 0
                while(_ind <= 15): # wait 45 sec fro receiving the code
                    time.sleep(3) # every iteration waiting 3 sec

                    code = re.findall(r"验证码：(\d+)，", sms.getMsg(self.phone))
                    # code = ['883568']
                    if(len(code) > 0): # if receive the code 
                        try:
                            driver.find_element_by_xpath('//input[@class="j-inputtext pcin"]').send_keys(code[0]) # input the code
                        except BaseException as e:
                            return False
                        break
                    _ind += 1
                # WebDriverWait(driver, 100).until(WaitValueMatch((By.CLASS_NAME, 'j-inputtext'), r'[0-9]+'))
                if(_ind > 15): # did not receive the code after 120 sec
                    self.blockPhone()
                    return False
                # time.sleep(1) #sleep for clicking the login button
                # try:
                #     driver.find_element_by_xpath('//div[@class="f-cb loginbox"]/a').click()
                # except BaseException as e:
                #     return False
            try:
                driver.switch_to.default_content()

                driver.find_element_by_xpath('//div[@id="remember-me"]/span/i').click() # click 10 days no login

                time.sleep(1)

                driver.switch_to.frame(driver.find_element_by_xpath('//div[@id="j_login"]/iframe')) # swith to the login iframe

                driver.find_element_by_xpath('//div[@class="f-cb loginbox"]/a').click() # click login

                driver.switch_to.default_content()
                
                WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'go_to_steam'))) # wait maxmium 30 sec to login and get the cookie 这个需要修改下
            except BaseException as e:
                print('did not find go to steam button after login, account phone is ' + self.phone + ': ' + e + '\n')
                return False
            
            self.login_time = time.time()
            self.cookie, self.csrf_token = self.setCookieAndToken(driver.get_cookies())
            self.status = self.accountStatus()
            
            if self.status < 0:
                self.blockPhone()
                return False
            return True
        except BaseException as e:
            print('login exception: ' + e)
            return False

In [7]:
viewer = Viewer()

In [8]:
viewer.login()

36.213.73.168:16816


TypeError: must be str, not SessionNotCreatedException

In [5]:
class Buyer(Account):
    '''
    methods:
        super from Account class:
        1. isIpAlive: check the ip banded with the account is alive or not. return true or false.
        2. accountStatus: check the status of the account, 0: logined and market available. 1: did not login. -1: can not access market or other problems  ip problem or account blocked problem.
        3. setCookieAndToken: process cookie map to cookie str and token str. return cookie, csrf_token.
        4. updateIp: update ip.
        5. getIp: return the new ip.
        
        self functions:
        1. login: login account. return true or false.
        2. blockPhone: None: block the phone
    '''
    
    def blockPhone(self) -> None:
        '''
            block the useless phone.
        '''
        buyerlist.disableAccount(self.phone)
        
        print('block buyer phone: ' + self.phone)
    
    def login(self) -> bool:
        '''
            login the new or expired account, return the bool type as success or not.
            process:
                1. open the page and click the login/register button.
                2. wait 1.5s to open the login iframe page.
                3. click 10 days no login and the agreement.
                4. get the phone number and input into the phone input box automatically.
                5. click the send sms button, and wait 5 sec for the HUMAN VERIFICATION. (DO IT BY SELF!!!)
                6. if did not receive the sms after 45 sec, block the phone number and return false.
                7. click the login button and wait maximum 30 sec to login and get the cookies.
                8. check the account status, return true if logined or vice vera.
        '''
        try:

            options =  webdriver.ChromeOptions()

            options.add_experimental_option("detach", True) # make the chrome window always opened
            
            
            while True: # get the useful ip
                print(self.ip)
                options.add_argument(f"--proxy-server={self.ip}")
                # driver = webdriver.Chrome(executable_path= '/Users/yin/Desktop/chromedriver-mac-x64/chromedriver', options = options)
                driver = webdriver.Chrome(options = options)
                driver.get(Constant.ip_check_addr)
                try:
                    return_ip_addr = driver.find_element_by_tag_name('pre') # return 101.37.22.207
                    if return_ip_addr.text == self.ip.split(":")[0]:
                        print('IP ' + self.ip + ' can be use' + '\n')
                        driver.close()
                        break
                except BaseException as e:
                    driver.close()
                    options.arguments.remove(f"--proxy-server={self.ip}")
                    self.updateIp()
                    continue
            
            
            # driver = webdriver.Chrome(executable_path= '/Users/yin/Desktop/chromedriver-mac-x64/chromedriver', options = options)
            driver = webdriver.Chrome(options = options)

            driver.get(Constant.domain)
            try:
                driver.find_element_by_xpath('//div[@class="nav nav_entries"]/ul/li/a[@onclick="loginModule.showLogin()"]').click() # click login/register button
            except BaseException as e:
                return False

            try:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@id="j_login"]/iframe')))

                driver.switch_to.frame(driver.find_element_by_xpath('//div[@id="j_login"]/iframe')) # swith to the login iframe

                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//input[@id="phoneipt"]')))

            except BaseException as e:
                return False
            
            try:
                driver.find_element_by_xpath('//a[@class="tab0"]').click() # click login with password
                
                time.sleep(1)

                # driver.find_element_by_xpath('//div[@id="agree-checkbox"]/span/i').click() # click agreement
                driver.find_element_by_xpath('//input[@class="dl-clause-login"]').click() # click agreement

                driver.find_element_by_xpath('//input[@id="phoneipt"]').send_keys(self.BUFF_ACCOUNT)  # input phone number

                driver.find_element_by_xpath('//input[@class="j-inputtext dlemail"]').send_keys(self.BUFF_PWD)  # input password

            except BaseException as e:
                return False
            
            # code = []
            # while(len(code) == 0): # did not receive the code verification
            #     self.phone = sms.getPhone() # get a new phone number
            #     # self.phone = '16537928820'
            #     print('buff phone number: ' + self.phone)
            #     try:
            #         driver.find_element_by_xpath('//input[@class="dlemail j-nameforslide"]').send_keys(self.phone) 
            #     except BaseException as e:
            #         return  False
            #     time.sleep(5) # time sleep for the human verification
            #     try:
            #         driver.find_element_by_xpath('//div[@class="pcbtn f-fl"]/a').click() # send the sms code
            #     except BaseException as e:
            #         return False
            #     _ind = 0
            #     while(_ind <= 15): # wait 45 sec fro receiving the code
            #         time.sleep(3) # every iteration waiting 3 sec
# 
            #         code = re.findall(r"验证码：(\d+)，", sms.getMsg(self.phone))
            #         # code = ['883568']
            #         if(len(code) > 0): # if receive the code 
            #             try:
            #                 driver.find_element_by_xpath('//input[@class="j-inputtext pcin"]').send_keys(code[0]) # input the code
            #             except BaseException as e:
            #                 return False
            #             break
            #         _ind += 1
            #     # WebDriverWait(driver, 100).until(WaitValueMatch((By.CLASS_NAME, 'j-inputtext'), r'[0-9]+'))
            #     if(_ind > 15): # did not receive the code after 120 sec
            #         self.blockPhone()
            #         return False
            #     # time.sleep(1) #sleep for clicking the login button
            #     try:
            #         driver.find_element_by_xpath('//div[@class="f-cb loginbox"]/a').click()
            #     except BaseException as e:
            #         return False
            try:
                driver.switch_to.default_content()

                driver.find_element_by_xpath('//div[@id="remember-me"]/span/i').click() # click 10 days no login

                time.sleep(1)

                driver.switch_to.frame(driver.find_element_by_xpath('//div[@id="j_login"]/iframe')) # swith to the login iframe

                driver.find_element_by_xpath('//a[@id="submitBtn"]').click() # click login

                driver.switch_to.default_content()
                
                WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'go_to_steam'))) # wait maxmium 30 sec to login and get the cookie 这个需要修改下
            except BaseException as e:
                print('did not find go to steam button after login, account phone is ' + self.phone + ': ' + e + '\n')
                return False
            
            self.login_time = time.time()
            self.cookie, self.csrf_token = self.setCookieAndToken(driver.get_cookies())
            self.status = self.accountStatus()
            
            if self.status < 0:
                self.blockPhone()
                return False
            return True
        except BaseException as e:
            print('login exception: ' + e)
            return False

In [3]:
buyer = Buyer()

In [4]:
buyer.login()

36.213.73.168:16816
IP 36.213.73.168:16816 can be use

1
2
3
check account status, the code is: OK



True

In [11]:
buyerlist.buyer

,BUFF_ACCOUNT,BUFF_PWD,STEAM_EMAIL,STEAM_ACCOUNT,STEAM_PWD
0,17706619337,Txjhnnb12345,qiyao.yin@hotmail.com,jerryin11,yqy12345
1,17056635880,Yqy@12345,kyiaqmhoj6@suiyongsuiqi.com,jerryin5,txjhnnb12345
2,16737138654,Yqy@123456,591lmbcdvlg@suiyongsuiqi.com,jerryin7,txjhnnb12345
3,16571003870,Yqy@1234567,m33rb7ot31i@suiyongsuiqi.com,jerryin8,txjhnnb12345
4,17050573086,Yqy@12345678,nctux7if4ge@suiyongsuiqi.com,jerryin9,txjhnnb12345
5,16215707729,Yqy@123456789,g4hedeg664@suiyongsuiqi.com,jerryin10,yqy12345


In [12]:
#class Account(object):
#    '''
#    methods:
#        1. isIpAlive: check the ip banded with the account is alive or not. return true or false.
#        2. accountStatus: check the status of the account, 0: logined and market available. 1: did not login. -1: can not access market or other problems  ip problem or account blocked problem.
#        3. setCookieAndToken: process cookie map to cookie str and token str. return cookie, csrf_token.
#        4. updateIp: update ip.
#        5. getIp: return the new ip.
#    '''
#    def __init__(self) -> None:
#        self.ip = self.getIp() # like 127.0.0.1
#        self.phone = '' 
#        # self.login_time = 0
#        self.cookie = ''
#        self.csrf_token = '' # the token for visiting website
#        self.status = -1
#    
#    def isIpAlive(self) -> bool:
#        '''
#            check the ip banded with the account is alive or not.
#        '''
#        return ip.checkIpAlive(self.ip)
#    
#    def accountStatus(self) -> int:
#        '''
#        return: 
#            0: login and market available
#            1: did not login
#            -1: can not access market or other problems  ip problem or account blocked problem。
#        '''
#        try:
#            headers = {
#                ':method': 'GET',
#                ':path': Constant.account_availability_check_re_addr,
#                'cookie': self.cookie + 'csrf_token=' + self.csrf_token,
#                'sec-fetch-user': '?1',
#                'upgrade-insecure-requests': '1',
#            }
#
#            tmp_session = Constant.getSession() # get a tmp session
#            tmp_session.proxies.update(Constant.getProxy(self.ip))
#            
#            resp = Constant.sessions.get(Constant.account_availability_check_ab_addr, headers = headers)
#            code = resp.json()['code']
#            print('check account status, the code is: ' + code + '\n')
#            if code == 'OK' or code == 'ok':
#                return 0
#            elif code == 'Login Required':
#                return 1
#            else:
#                return -1
#        except:
#            return -1
#    
#    def setCookieAndToken(self, cookieList: list) -> tuple: 
#        '''
#            process cookie map to cookie str and token str.
#        '''
#        cookie_map = cookieList
#        cookie = ''
#        deviceId = ''
#        LocaleSupported = ''
#        game = ''
#        NTES_YD_SESS = ''
#        S_INFO = ''
#        P_INFO = ''
#        remember_me = ''
#        session = ''
#        csrf_token = ''
#        for _map in cookie_map:
#            if(_map['name'] == 'csrf_token'):
#                csrf_token = _map['value']
#            elif(_map['name'] == 'Device-Id'):
#                deviceId = 'Device-Id=' + _map['value'] + '; '
#            elif(_map['name'] == 'Locale-Supported'):
#                LocaleSupported = 'Locale-Supported=' + _map['value'] + '; '
#            elif(_map['name'] == 'game'):
#                game = 'game=' + _map['value'] + '; '
#            elif(_map['name'] == 'NTES_YD_SESS'):
#                NTES_YD_SESS = 'NTES_YD_SESS=' + _map['value'] + '; '
#            elif(_map['name'] == 'S_INFO'):
#                S_INFO ='S_INFO=' + _map['value'] + '; '
#            elif(_map['name'] == 'P_INFO'):
#                P_INFO = 'P_INFO=' + _map['value'] + '; '
#            elif(_map['name'] == 'remember_me'):
#                remember_me = 'remember_me=' + _map['value'] + '; '
#            elif(_map['name'] == 'session'):
#                session = 'session=' + _map['value'] + '; '
#        cookie = deviceId + LocaleSupported + game + NTES_YD_SESS + S_INFO + P_INFO + remember_me + session
#        return cookie, csrf_token
#    
#    def updateIp(self) -> None:
#        '''
#            update the ip banded with this account.
#        '''
#        self.ip = self.getIp()
#    
#    def getIp(self) -> str:
#        '''
#            return the new ip.
#        '''
#        return ip.getIp()
#    